In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
! pip install -q gamspy
import numpy as np
import pandas as pd
import sys
from gamspy import Container, Set, Parameter, Variable, Equation, Model, Sum, Sense, Alias

Mounted at /content/gdrive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 27.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tenso

In [2]:
cost=pd.DataFrame([["1","4",1],["4","5",-4],["2","4",6],["4","6",-9]],columns=["Source","Destination","Cost"]).set_index(["Source","Destination"])



capacity=pd.DataFrame([["1","4",4],["4","5",3],["2","4",7],["4","6",4]],columns=["Source","Destination","Cost"]).set_index(["Source","Destination"])

bien_d=pd.DataFrame([["1",4],["2",1],["4",0],["5",-3],["6",-2]],columns=["Name","Value of d"]).set_index("Name")


In [3]:
m=Container()
node_name=Set(m,name="Node_i",records=bien_d.index)

j=Alias(m,name="Node_j",alias_with=node_name)


In [4]:
u=Parameter(m,name="u",domain=[j,node_name],description="Full capacity",records=capacity.reset_index())

g=Parameter(m,name="g",domain=[node_name,j],description="Generalized cost",records=cost.reset_index())

d=Parameter(m,name="d",domain=[node_name],description="Capac of the node",records=bien_d.reset_index())


# print(u.records)
# print(g.records)
# print(d.records)
# print(u.hasDomainViolations())
# print(g.hasDomainViolations())

In [5]:
x=Variable(m,name="x",domain=[node_name,j],type="integer",description="Variable needed")


eq=Equation(m,name="Balance",domain=[node_name],description="Balance equ")
eq1=Equation(m,name="Constrain",domain=[node_name,j],description="Constr equ")
eq2=Equation(m,name="Constrain1",domain=[node_name,j],description="Constr equ1")

eq[node_name]=Sum(j,x[node_name,j]-x[j,node_name])==d[node_name]
eq1[node_name,j]=x[node_name,j]<=u[node_name,j]
eq2[node_name,j]=x[node_name,j]>=0

In [6]:
pro=Model(m,name="Sub_1",equations=[eq,eq1,eq2],problem="MIP",sense=Sense.MIN,objective=Sum((node_name,j),g[node_name,j]*x[node_name,j]))

In [7]:
pro.solve(output=sys.stdout)

--- Job _job_d2c4470a-cfea-4fda-aa5f-6d0ca2d36cd2.gms Start 12/28/23 14:08:16 45.5.0 8d95ac18 LEX-LEG x86 64bit/Linux
--- Applying:
    /usr/local/lib/python3.10/dist-packages/gamspy_base/gmsprmun.txt
--- GAMS Parameters defined
    LP CPLEX
    MIP CPLEX
    RMIP CPLEX
    NLP CONOPT
    MCP PATH
    MPEC NLPEC
    RMPEC CONVERT
    CNS CONOPT
    DNLP CONOPT
    RMINLP CONOPT
    MINLP SBB
    QCP CONOPT
    MIQCP SBB
    RMIQCP CONOPT
    EMP CONVERT
    Restart /tmp/tmpcm_21qfu/_restart_a07d7aa4-9d8b-4e57-9d83-c1aa3fde010b.g00
    Input /tmp/tmpcm_21qfu/_job_d2c4470a-cfea-4fda-aa5f-6d0ca2d36cd2.gms
    Output /tmp/tmpcm_21qfu/_job_d2c4470a-cfea-4fda-aa5f-6d0ca2d36cd2.lst
    Save /tmp/tmpcm_21qfu/_save_a07d7aa4-9d8b-4e57-9d83-c1aa3fde010b.g00
    ScrDir /tmp/tmpcm_21qfu/225a/
    SysDir /usr/local/lib/python3.10/dist-packages/gamspy_base/
    CurDir /tmp/tmpcm_21qfu/
    LogOption 3
    LogFile /tmp/tmpcm_21qfu/_job_d2c4470a-cfea-4fda-aa5f-6d0ca2d36cd2.log
    Trace /tmp/tmpcm_21qf

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,-20,56,26,MIP,CPLEX,0.008


In [8]:
print(x.records)
print(pro.objective_value)

   Node_i Node_j  level  marginal  lower  upper  scale
0       1      1    0.0      -0.0    0.0    inf    1.0
1       1      2    0.0      -0.0    0.0    inf    1.0
2       1      4    4.0       1.0    0.0    inf    1.0
3       1      5    0.0      -0.0    0.0    inf    1.0
4       1      6    0.0      -0.0    0.0    inf    1.0
5       2      1    0.0      -0.0    0.0    inf    1.0
6       2      2    0.0      -0.0    0.0    inf    1.0
7       2      4    1.0       6.0    0.0    inf    1.0
8       2      5    0.0      -0.0    0.0    inf    1.0
9       2      6    0.0      -0.0    0.0    inf    1.0
10      4      1    0.0      -0.0    0.0    inf    1.0
11      4      2    0.0      -0.0    0.0    inf    1.0
12      4      4    0.0      -0.0    0.0    inf    1.0
13      4      5    3.0      -4.0    0.0    inf    1.0
14      4      6    2.0      -9.0    0.0    inf    1.0
15      5      1    0.0      -0.0    0.0    inf    1.0
16      5      2    0.0      -0.0    0.0    inf    1.0
17      5 